In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import joblib

: 

In [52]:
df = pd.read_csv('TrafficTwoMonth.csv')

In [53]:
df['Time_hour'] = pd.to_datetime(df['Time']).dt.hour
df['Time_minute'] = pd.to_datetime(df['Time']).dt.minute

df['Day_of_the_week_encoded'] = df['Day of the week'].map({
    'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6
})

print(df[['Time', 'Time_hour', 'Time_minute', 'Day of the week', 'Day_of_the_week_encoded']].head())

          Time  Time_hour  Time_minute Day of the week  \
0  12:00:00 AM          0            0         Tuesday   
1  12:15:00 AM          0           15         Tuesday   
2  12:30:00 AM          0           30         Tuesday   
3  12:45:00 AM          0           45         Tuesday   
4   1:00:00 AM          1            0         Tuesday   

   Day_of_the_week_encoded  
0                        1  
1                        1  
2                        1  
3                        1  
4                        1  


C:\Users\XPS\AppData\Local\Temp\ipykernel_16736\471517607.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time_hour'] = pd.to_datetime(df['Time']).dt.hour
C:\Users\XPS\AppData\Local\Temp\ipykernel_16736\471517607.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time_minute'] = pd.to_datetime(df['Time']).dt.minute


In [54]:
label_encoder = LabelEncoder()
df['Traffic_Situation_encoded'] = label_encoder.fit_transform(df['Traffic Situation'])
y_traffic_situation = to_categorical(df['Traffic_Situation_encoded']) 

vehicle_count_features = ['CarCount', 'BikeCount', 'BusCount', 'TruckCount']
y_vehicle_counts = df[vehicle_count_features].values

numerical_features = ['Time_hour', 'Time_minute', 'Date', 'Day_of_the_week_encoded']

print(f"Shape of y_traffic_situation (one-hot encoded): {y_traffic_situation.shape}")
print(f"Shape of y_vehicle_counts (raw counts): {y_vehicle_counts.shape}")
print(f"New numerical input features: {numerical_features}")

Shape of y_traffic_situation (one-hot encoded): (5952, 4)
Shape of y_vehicle_counts (raw counts): (5952, 4)
New numerical input features: ['Time_hour', 'Time_minute', 'Date', 'Day_of_the_week_encoded']


In [55]:
scaler = MinMaxScaler()

X_numerical_scaled = scaler.fit_transform(df[numerical_features])
X_numerical_reshaped = X_numerical_scaled.reshape(X_numerical_scaled.shape[0], X_numerical_scaled.shape[1], 1)

print(f"Shape of X_numerical_reshaped (for CNN input): {X_numerical_reshaped.shape}")

Shape of X_numerical_reshaped (for CNN input): (5952, 4, 1)


In [56]:
onehot_encoder_location = OneHotEncoder(sparse_output=False)
X_location_onehot = onehot_encoder_location.fit_transform(df[['Location']])

print(f"Shape of X_location_onehot (for Location input): {X_location_onehot.shape}")
print(f"Unique locations encoded: {onehot_encoder_location.categories_[0]}")

Shape of X_location_onehot (for Location input): (5952, 2)
Unique locations encoded: ['Delhi' 'Mumbai']


In [57]:
X_train_num, X_test_num, \
X_train_loc, X_test_loc, \
y_train_traffic, y_test_traffic, \
y_train_vehicles, y_test_vehicles = train_test_split(
    X_numerical_reshaped, X_location_onehot, y_traffic_situation, y_vehicle_counts,
    test_size=0.2, random_state=42
)

print(f"X_train_num shape: {X_train_num.shape}, X_test_num shape: {X_test_num.shape}")
print(f"X_train_loc shape: {X_train_loc.shape}, X_test_loc shape: {X_test_loc.shape}")
print(f"y_train_traffic shape: {y_train_traffic.shape}, y_test_traffic shape: {y_test_traffic.shape}")
print(f"y_train_vehicles shape: {y_train_vehicles.shape}, y_test_vehicles shape: {y_test_vehicles.shape}")

X_train_num shape: (4761, 4, 1), X_test_num shape: (1191, 4, 1)
X_train_loc shape: (4761, 2), X_test_loc shape: (1191, 2)
y_train_traffic shape: (4761, 4), y_test_traffic shape: (1191, 4)
y_train_vehicles shape: (4761, 4), y_test_vehicles shape: (1191, 4)


In [58]:
joblib.dump(scaler, 'traffic_scaler_temporal_spatial_only.pkl') 
joblib.dump(label_encoder, 'traffic_label_encoder_temporal_spatial_only.pkl') 
joblib.dump(onehot_encoder_location, 'location_onehot_encoder_temporal_spatial_only.pkl')

print("NEW Preprocessing objects saved with updated filenames.")

NEW Preprocessing objects saved with updated filenames.


In [59]:
numerical_input = Input(shape=(X_train_num.shape[1], 1), name='numerical_input')
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(numerical_input)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling1D(pool_size=2)(batch1)
drop1 = Dropout(0.3)(pool1)

conv2 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(drop1)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling1D(pool_size=2)(batch2)
drop2 = Dropout(0.3)(pool2)

flatten = Flatten()(drop2)

location_input = Input(shape=(X_train_loc.shape[1],), name='location_input')
location_dense = Dense(32, activation='relu')(location_input)

merged = concatenate([flatten, location_dense])

dense1 = Dense(128, activation='relu')(merged)
batch3 = BatchNormalization()(dense1)
drop3 = Dropout(0.4)(batch3)

dense2 = Dense(64, activation='relu')(drop3)
batch4 = BatchNormalization()(dense2)
drop4 = Dropout(0.4)(batch4)

traffic_situation_output = Dense(
    y_traffic_situation.shape[1], activation='softmax', name='traffic_situation_output'
)(drop4)

vehicle_counts_output = Dense(
    len(vehicle_count_features), activation='relu', name='vehicle_counts_output'
)(drop4)

model = Model(
    inputs=[numerical_input, location_input],
    outputs=[traffic_situation_output, vehicle_counts_output]
)

In [61]:
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss={
        'traffic_situation_output': 'categorical_crossentropy', 
        'vehicle_counts_output': 'mse'                         
    },
    metrics={
        'traffic_situation_output': ['accuracy'],             
        'vehicle_counts_output': ['mae']                       
    }
)

print("Model compiled successfully.")

Model compiled successfully.


In [62]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ numerical_input     │ (None, 4, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 4, 64)     │        256 │ numerical_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 64)     │        256 │ conv1d_6[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_6     │ (None, 2, 64)     │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 2, 64)     │          0 │ max_pooling1d_6[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 2, 128)    │     24,704 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2, 128)    │        512 │ conv1d_7[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_7     │ (None, 1, 128)    │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 1, 128)    │          0 │ max_pooling1d_7[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ location_input      │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 128)       │          0 │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │         96 │ location_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 160)       │          0 │ flatten_3[0][0],  │
│ (Concatenate)       │                   │            │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 128)       │     20,608 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_10[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 64)        │      8,256 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_11[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 64)        │          0 │ batch_normalizat

 Total params: 55,976 (218.66 KB)

 Trainable params: 55,208 (215.66 KB)

 Non-trainable params: 768 (3.00 KB)

In [63]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("Starting model training...")
history = model.fit(
    {'numerical_input': X_train_num, 'location_input': X_train_loc},
    {'traffic_situation_output': y_train_traffic, 'vehicle_counts_output': y_train_vehicles},
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)
print("Model training completed.")

Starting model training...
Epoch 1/100


119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 1769.3263 - traffic_situation_output_accuracy: 0.3036 - traffic_situation_output_loss: 2.0387 - vehicle_counts_output_loss: 1767.2875 - vehicle_counts_output_mae: 26.4374 - val_loss: 1450.4517 - val_traffic_situation_output_accuracy: 0.5929 - val_traffic_situation_output_loss: 1.1772 - val_vehicle_counts_output_loss: 1449.3038 - val_vehicle_counts_output_mae: 23.4862
Epoch 2/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1520.7039 - traffic_situation_output_accuracy: 0.3582 - traffic_situation_output_loss: 1.6251 - vehicle_counts_output_loss: 1519.0785 - vehicle_counts_output_mae: 23.8640 - val_loss: 817.5380 - val_traffic_situation_output_accuracy: 0.5247 - val_traffic_situation_output_loss: 1.2165 - val_vehicle_counts_output_loss: 816.4246 - val_vehicle_counts_output_mae: 20.3242
Epoch 3/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1188.5327 - traffic_situation_output_accuracy: 0.4013 - traffic_situation_output_loss: 1.4

In [64]:
print("Evaluating model on test set...")
results = model.evaluate(
    {'numerical_input': X_test_num, 'location_input': X_test_loc},
    {'traffic_situation_output': y_test_traffic, 'vehicle_counts_output': y_test_vehicles},
    verbose=0
)

traffic_loss = results[0]
vehicle_loss_mse = results[1]
traffic_accuracy = results[2]
vehicle_mae = results[3]

print(f"\n--- Model Evaluation on Test Set ---")
print(f"Traffic Situation - Loss: {traffic_loss:.4f}, Accuracy: {traffic_accuracy:.4f}")
print(f"Vehicle Counts - Loss (MSE): {vehicle_loss_mse:.4f}, MAE: {vehicle_mae:.4f}")

Evaluating model on test set...

--- Model Evaluation on Test Set ---
Traffic Situation - Loss: 145.5997, Accuracy: 143.8672
Vehicle Counts - Loss (MSE): 0.7370, MAE: 0.7145


In [65]:
print("Making predictions on test set...")
predictions_traffic, predictions_vehicles = model.predict(
    {'numerical_input': X_test_num, 'location_input': X_test_loc}
)

predicted_traffic_classes = np.argmax(predictions_traffic, axis=1)
predicted_traffic_labels = label_encoder.inverse_transform(predicted_traffic_classes)

true_traffic_classes = np.argmax(y_test_traffic, axis=1)
true_traffic_labels = label_encoder.inverse_transform(true_traffic_classes)

print("\nSample of predicted vs true Traffic Situation:")
for i in range(10):
    print(f"True: {true_traffic_labels[i]}, Predicted: {predicted_traffic_labels[i]}")

print("\nSample of predicted vs true Vehicle Counts:")
print("Predicted (first 5):")
print(predictions_vehicles[:5].round(0))
print("True (first 5):")
print(y_test_vehicles[:5])

Making predictions on test set...
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

Sample of predicted vs true Traffic Situation:
True: low, Predicted: normal
True: normal, Predicted: heavy
True: normal, Predicted: normal
True: high, Predicted: normal
True: normal, Predicted: heavy
True: normal, Predicted: normal
True: heavy, Predicted: heavy
True: low, Predicted: normal
True: normal, Predicted: normal
True: normal, Predicted: normal

Sample of predicted vs true Vehicle Counts:
Predicted (first 5):
[[ 17.   2.   0.  24.]
 [110.  22.  22.   6.]
 [ 49.   9.  13.  21.]
 [ 72.  16.  23.  13.]
 [110.  19.  20.  12.]]
True (first 5):
[[ 19   1   0  14]
 [117  10  10  18]
 [ 52   7   1  26]
 [ 80  26  35   6]
 [104  17  23   5]]


In [66]:
print("\n--- Traffic Situation Classification Report ---")
print(classification_report(true_traffic_labels, predicted_traffic_labels))

print("\n--- Traffic Situation Confusion Matrix ---")
print(confusion_matrix(true_traffic_labels, predicted_traffic_labels))


--- Traffic Situation Classification Report ---
              precision    recall  f1-score   support

       heavy       0.66      0.81      0.73       216
        high       0.00      0.00      0.00        81
         low       0.00      0.00      0.00       146
      normal       0.73      0.91      0.81       748

    accuracy                           0.71      1191
   macro avg       0.35      0.43      0.38      1191
weighted avg       0.58      0.71      0.64      1191


--- Traffic Situation Confusion Matrix ---
[[174   0   0  42]
 [ 13   0   0  68]
 [  4   0   0 142]
 [ 71   0   0 677]]


c:\Users\XPS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\XPS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\XPS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [67]:
print("\n--- Vehicle Counts Regression Metrics (MAE per vehicle type) ---")
for i, col in enumerate(vehicle_count_features):
    mse = mean_squared_error(y_test_vehicles[:, i], predictions_vehicles[:, i])
    mae = mean_absolute_error(y_test_vehicles[:, i], predictions_vehicles[:, i])
    print(f"{col} - MSE: {mse:.2f}, MAE: {mae:.2f}")


--- Vehicle Counts Regression Metrics (MAE per vehicle type) ---
CarCount - MSE: 414.03, MAE: 14.99
BikeCount - MSE: 45.44, MAE: 4.97
BusCount - MSE: 61.86, MAE: 5.59
TruckCount - MSE: 58.12, MAE: 5.96


In [68]:
model.save('traffic_cnn_temporal_spatial_only_model.h5')
print("NEW multi-output model saved as 'traffic_cnn_temporal_spatial_only_model.h5'.")

NEW multi-output model saved as 'traffic_cnn_temporal_spatial_only_model.h5'.


In [69]:
def load_traffic_model_with_location():
    """
    Loads the updated traffic prediction model and preprocessing objects.
    """
    model = tf.keras.models.load_model('traffic_cnn_multi_output_model.h5')
    scaler = joblib.load('traffic_scaler_with_location_updated.pkl')
    label_encoder = joblib.load('traffic_label_encoder_with_location_updated.pkl')
    onehot_encoder_location = joblib.load('location_onehot_encoder_updated.pkl')
    return model, scaler, label_encoder, onehot_encoder_location

print("Load function defined.")

Load function defined.


In [70]:
def predict_traffic_and_counts(model, scaler, label_encoder, onehot_encoder_location,
                               time_str, date_day, day_of_week_str,
                               car_count_current, bike_count_current, bus_count_current, truck_count_current,
                               total_count_current, location_str):

    input_data = pd.DataFrame([{
        'Time': time_str,
        'Date': date_day,
        'Day of the week': day_of_week_str,
        'CarCount': car_count_current,
        'BikeCount': bike_count_current,
        'BusCount': bus_count_current,
        'TruckCount': truck_count_current,
        'Total': total_count_current,
        'Location': location_str
    }])

    input_data['Time_hour'] = pd.to_datetime(input_data['Time']).dt.hour
    input_data['Time_minute'] = pd.to_datetime(input_data['Time']).dt.minute

    day_of_week_mapping = {'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6}
    input_data['Day_of_the_week_encoded'] = input_data['Day of the week'].map(day_of_week_mapping)

    numerical_features_for_scaling = ['CarCount', 'BikeCount', 'BusCount', 'TruckCount', 'Total',
                                      'Time_hour', 'Time_minute', 'Date', 'Day_of_the_week_encoded']
    X_new_numerical_scaled = scaler.transform(input_data[numerical_features_for_scaling])
    X_new_numerical_reshaped = X_new_numerical_scaled.reshape(1, X_new_numerical_scaled.shape[1], 1)

    X_new_location_onehot = onehot_encoder_location.transform(input_data[['Location']])

    pred_traffic, pred_vehicles = model.predict([X_new_numerical_reshaped, X_new_location_onehot])

    predicted_traffic_class_idx = np.argmax(pred_traffic, axis=1)[0]
    predicted_traffic_situation = label_encoder.inverse_transform([predicted_traffic_class_idx])[0]

    predicted_vehicle_counts = pred_vehicles[0].round(0).astype(int) 

    return predicted_traffic_situation, predicted_vehicle_counts[0], predicted_vehicle_counts[1], \
           predicted_vehicle_counts[2], predicted_vehicle_counts[3]
